# Process Uber trip data

This is the process of reading monthly PARQUET trip files, selected the columns of interest, and converting to a concatenated, one-year CSV file. This will be used to create the dashboard later.


In [1]:
from fastparquet import ParquetFile

In [2]:
pf1 = ParquetFile("fhvhv_tripdata_2022-01.parquet")
df1 = pf1.to_pandas(
    [
        "hvfhs_license_num",
        "request_datetime",
        "PULocationID",
        "DOLocationID",
        "trip_miles",
        "trip_time",
        "base_passenger_fare",
        "tolls",
        "bcf",
        "sales_tax",
        "congestion_surcharge",
        "airport_fee",
        "tips",
        "driver_pay",
    ]
)
df1.head()

,hvfhs_license_num,request_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay
0,HV0003,2022-01-01 00:05:31,170,161,1.18,664,24.90,0.0,0.75,2.21,2.75,0.0,0.0,23.03
1,HV0003,2022-01-01 00:19:27,237,161,0.82,460,11.97,0.0,0.36,1.06,2.75,0.0,0.0,12.32
2,HV0003,2022-01-01 00:43:53,237,161,1.18,595,29.82,0.0,0.89,2.65,2.75,0.0,0.0,23.30
3,HV0003,2022-01-01 00:15:36,262,229,1.65,303,7.91,0.0,0.24,0.70,2.75,0.0,0.0,6.30
4,HV0003,2022-01-01 00:25:45,229,141,1.65,461,9.44,0.0,0.28,0.84,2.75,0.0,0.0,7.44


Based on the data dictionary file provided with these datasets, Uber has the HVFHS code of `HV0003`. Filter the dataset to only contain rows corresponding to Uber.


In [3]:
df1 = df1[df1["hvfhs_license_num"] == "HV0003"]
df1 = df1[
    [
        "request_datetime",
        "PULocationID",
        "DOLocationID",
        "trip_miles",
        "trip_time",
        "base_passenger_fare",
        "tolls",
        "bcf",
        "sales_tax",
        "congestion_surcharge",
        "airport_fee",
        "tips",
        "driver_pay",
    ]
]
df1.head()

,request_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay
0,2022-01-01 00:05:31,170,161,1.18,664,24.90,0.0,0.75,2.21,2.75,0.0,0.0,23.03
1,2022-01-01 00:19:27,237,161,0.82,460,11.97,0.0,0.36,1.06,2.75,0.0,0.0,12.32
2,2022-01-01 00:43:53,237,161,1.18,595,29.82,0.0,0.89,2.65,2.75,0.0,0.0,23.30
3,2022-01-01 00:15:36,262,229,1.65,303,7.91,0.0,0.24,0.70,2.75,0.0,0.0,6.30
4,2022-01-01 00:25:45,229,141,1.65,461,9.44,0.0,0.28,0.84,2.75,0.0,0.0,7.44


Create a CSV file first so the other months can be concatenated to it. This is done by a loop basically repeating the steps demonstrated above.


In [4]:
df1.to_csv("tripdata_all.csv", index=False)

In [5]:
for i in range(2, 13):
    if i < 10:
        pf = ParquetFile("fhvhv_tripdata_2022-0{}.parquet".format(i))
    else:
        pf = ParquetFile("fhvhv_tripdata_2022-{}.parquet".format(i))
    df = pf.to_pandas(
        [
            "hvfhs_license_num",
            "request_datetime",
            "PULocationID",
            "DOLocationID",
            "trip_miles",
            "trip_time",
            "base_passenger_fare",
            "tolls",
            "bcf",
            "sales_tax",
            "congestion_surcharge",
            "airport_fee",
            "tips",
            "driver_pay",
        ]
    )
    df = df[df["hvfhs_license_num"] == "HV0003"]
    df = df[
        [
            "request_datetime",
            "PULocationID",
            "DOLocationID",
            "trip_miles",
            "trip_time",
            "base_passenger_fare",
            "tolls",
            "bcf",
            "sales_tax",
            "congestion_surcharge",
            "airport_fee",
            "tips",
            "driver_pay",
        ]
    ]
    df.to_csv("tripdata_all.csv", mode="a", index=False, header=False)
    del df